# Candlesticks patterns and statistical analysis
# Loading and preparing data

In [2]:
import pandas as pd
df = pd.read_csv("EURUSD_Candlestick_1_D_ASK_05.05.2003-19.10.2019.csv")
df.tail()

,Local time,open,high,low,close,volume
4322,14.10.2019 00:00:00.000 GMT+0300,1.10399,1.10428,1.10129,1.10281,150329.9600
4323,15.10.2019 00:00:00.000 GMT+0300,1.10281,1.10463,1.09915,1.10338,225114.4925
4324,16.10.2019 00:00:00.000 GMT+0300,1.10331,1.10857,1.10226,1.10720,224179.4392
4325,17.10.2019 00:00:00.000 GMT+0300,1.10724,1.11400,1.10653,1.11254,221922.6500
4326,18.10.2019 00:00:00.000 GMT+0300,1.11256,1.11727,1.11149,1.11723,143073.7750


In [3]:
#Check if any zero volumes are to be removed
indexZeros = df[ df['volume'] == 0 ].index

df.drop(indexZeros , inplace=True)
df.loc[(df["volume"] == 0 )]
df.isna().sum()

Local time    0
open          0
high          0
low           0
close         0
volume        0
dtype: int64

In [4]:
import numpy as np
import pandas_ta as ta
#df.ta.indicators()
#help(ta.atr)
df['ATR'] = df.ta.atr(length=10)
df['RSI'] = df.ta.rsi()
df.dropna()

,Local time,open,high,low,close,volume,ATR,RSI
14,23.05.2003 00:00:00.000 GMT+0300,1.16947,1.18373,1.16648,1.18266,1.050209e+06,0.013346,76.794368
15,26.05.2003 00:00:00.000 GMT+0300,1.18270,1.18754,1.17972,1.18690,1.052746e+06,0.012650,78.238516
16,27.05.2003 00:00:00.000 GMT+0300,1.18670,1.19300,1.18176,1.18288,1.057689e+06,0.012477,73.564081
17,28.05.2003 00:00:00.000 GMT+0300,1.18261,1.18453,1.17047,1.17649,1.060248e+06,0.012667,66.738430
18,29.05.2003 00:00:00.000 GMT+0300,1.17657,1.19008,1.16917,1.18833,1.045905e+06,0.013637,71.934617
...,...,...,...,...,...,...,...,...
4322,14.10.2019 00:00:00.000 GMT+0300,1.10399,1.10428,1.10129,1.10281,1.503300e+05,0.005444,53.542274
4323,15.10.2019 00:00:00.000 GMT+0300,1.10281,1.10463,1.09915,1.10338,2.251145e+05,0.005448,54.321918
4324,16.10.2019 00:00:00.000 GMT+0300,1.10331,1.10857,1.10226,1.10720,2.241794e+05,0.005534,59.256697
4325,17.10.2019 00:00:00.000 GMT+0300,1.10724,1.11400,1.10653,1.11254,2.219226e+05,0.005727,64.956163


In [5]:
# for down trends
# Shooting star

def Revsignal1(df1):
    df.dropna()
    df.reset_index(drop=True, inplace=True)
    
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    signal = [0] * length
    highdiff = [0] * length
    lowdiff = [0] * length
    bodydiff = [0] * length
    ratio1 = [0] * length
    ratio2 = [0] * length
        
    for row in range(0, length):
        
        highdiff[row] = high[row]-max(open[row],close[row])
        bodydiff[row] = abs(open[row]-close[row])
        if bodydiff[row]<0.002:
            bodydiff[row]=0.002
        lowdiff[row] = min(open[row],close[row])-low[row]
        ratio1[row] = highdiff[row]/bodydiff[row]
        ratio2[row] = lowdiff[row]/bodydiff[row]
        
        #print(df.RSI[row])
        #  |
        # _|_
        #|__|
        # |
        # 
        
        if (ratio1[row]>2.5 and lowdiff[row]<0.3*highdiff[row] and bodydiff[row]>0.03 and df.RSI[row]>50 and df.RSI[row]<70 ):
            signal[row] = 1
        
        #elif (ratio2[row-1]>2.5 and highdiff[row-1]<0.23*lowdiff[row-1] and bodydiff[row-1]>0.03 and bodydiff[row]>0.04 and close[row]>open[row] and close[row]>high[row-1] and df.RSI[row]<55 and df.RSI[row]>30):
        #    signal[row] = 2
        # _|_
        #|__|
        # |
        # |
        
        elif (ratio2[row]>2.5 and highdiff[row]<0.23*lowdiff[row] and bodydiff[row]>0.03 and df.RSI[row]<55 and df.RSI[row]>30):
            signal[row]=2
    return signal
df['signal1'] = Revsignal1(df)
df[df['signal1']==2].count()

Local time    0
open          0
high          0
low           0
close         0
volume        0
ATR           0
RSI           0
signal1       0
dtype: int64

In [6]:
#Target Shooting Star

def mytarget(barsupfront, df1):
    length = len(df1)
    high = list(df1['high'])
    low = list(df1['low'])
    close = list(df1['close'])
    open = list(df1['open'])
    datr = list(df1['ATR'])
    trendcat = [0] * length
    

    for line in range (0,length-barsupfront-1):
        valueOpenLow = 0
        valueOpenHigh = 0
        
        highdiff = high[line]-max(open[line],close[line])
        bodydiff = abs(open[line]-close[line])
        
        pipdiff = datr[line]*1. #highdiff*1.3 #for SL 400*1e-3
        if pipdiff<1.1:
            pipdiff=1.1
            
        SLTPRatio = 2. #pipdiff*Ratio gives TP
        
        for i in range(1,barsupfront+1):
            value1 = close[line]-low[line+i]
            value2 = close[line]-high[line+i]
            valueOpenLow = max(value1, valueOpenLow)
            valueOpenHigh = min(value2, valueOpenHigh)

            if ( (valueOpenLow >= (SLTPRatio*pipdiff) ) and (-valueOpenHigh < pipdiff) ):
                trendcat[line] = 1 #-1 downtrend
                break
            elif ((valueOpenLow < pipdiff) ) and (-valueOpenHigh >= (SLTPRatio*pipdiff)):
                trendcat[line] = 2 # uptrend
                break 
            else:
                trendcat[line] = 0 # no clear trend
            
    return trendcat

# mytarget(barsfront to take into account, dataframe)
df['Trend'] = mytarget(100, df)
df.tail(20)

,Local time,open,high,low,close,volume,ATR,RSI,signal1,Trend
4273,23.09.2019 00:00:00.000 GMT+0300,1.10181,1.10256,1.09664,1.09935,178252.8061,0.006594,41.504405,0,0
4274,24.09.2019 00:00:00.000 GMT+0300,1.09939,1.10244,1.09839,1.10203,180478.7978,0.006339,45.303727,0,0
4275,25.09.2019 00:00:00.000 GMT+0300,1.10203,1.10240,1.09376,1.09440,196215.9080,0.006569,37.780198,0,0
4276,26.09.2019 00:00:00.000 GMT+0300,1.09440,1.09675,1.09089,1.09227,190051.0630,0.006498,35.983674,0,0
4277,27.09.2019 00:00:00.000 GMT+0300,1.09231,1.09590,1.09049,1.09439,172543.0575,0.006390,39.088308,0,0
4278,30.09.2019 00:00:00.000 GMT+0300,1.09441,1.09481,1.08850,1.09008,168184.5779,0.006382,35.336276,0,0
4279,01.10.2019 00:00:00.000 GMT+0300,1.09004,1.09430,1.08793,1.09331,207316.7850,0.006380,39.985552,0,0
4280,02.10.2019 00:00:00.000 GMT+0300,1.09349,1.09638,1.09044,1.09594,202499.5268,0.006336,43.544864,0,0
4281,03.10.2019 00:00:00.000 GMT+0300,1.09594,1.09994,1.09411,1.09659,217992.0050,0.006286,44.422176,0,0
4282,04.10.2019 00:00:00.000 GMT+0300,1.09660,1.09995,1.09572,1.09815,186839.3200,0.006080,46.568255,0,0


In [7]:
conditions = [(df['Trend'] == 1) & (df['signal1'] == 1),
              (df['Trend'] == 2) & (df['signal1'] == 2)]
values = [1, 2]
df['result'] = np.select(conditions, values)

trendId=2
print(df[df['result']==trendId].result.count()/df[df['signal1']==trendId].signal1.count())
df[ (df['result']!=trendId) & (df['signal1']==trendId) ] # false positives

#histrs = df[ (df['result']==2) & (df['signal1']==2) ].RSI # false positives
#import matplotlib.pyplot as plt
#plt.hist(histrs,bins=2)  # density=False would make counts
#plt.ylabel('Probability')
#plt.xlabel('RSI');

nan


C:\Users\jacks\AppData\Local\Temp\ipykernel_165120\3370182367.py:7: RuntimeWarning: invalid value encountered in scalar divide
  print(df[df['result']==trendId].result.count()/df[df['signal1']==trendId].signal1.count())


,Local time,open,high,low,close,volume,ATR,RSI,signal1,Trend,result


In [8]:
dfpl = df[400:480]
import plotly.graph_objects as go
from datetime import datetime

fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

fig.show()